### EXAMEN - Convocatoria 1 - Programación
Utilizar el conjunto de datos "dataset_exam.npy" para resolver el ejercicio. Tener en cuenta que la última columna corresponde a la clase.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.svm import SVC

In [2]:
dataset = np.load('dataset_exam.npy')
X = dataset[:,:-1]
Y = dataset[:,-1]
print(np.unique(Y, return_counts=True))


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_exam.npy'

#### 1) Partición de datos externa (1 punto)
Realizar una partición externa de tipo hold-out seleccionando un 20% de los datos para test (fijar una semilla en 42).
Comprobad si los datos, tanto de train como de test, están más o menos balanceados.

In [ ]:
# realizar partición externa
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# comprobar que están balanceados
print(np.unique(Y_train, return_counts=True))
print(np.unique(Y_test, return_counts=True))

(array([0., 1.]), array([195, 198]))
(array([0., 1.]), array([51, 48]))


#### 2) Estandarización de los datos de train y test (1 punto)
Utilizar el método StandardScaler().

In [ ]:
# estandarización
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

#### 3) Selección de atributos en train y test (1 punto)
Aplicar el método de mutual information para clasificación seleccionando un percentile=90. (Recordad la función "SelectPercentile")

In [ ]:
selector = SelectPercentile(mutual_info_classif, percentile=90)
X_train_selected = selector.fit_transform(X_train_std, Y_train)
X_test_selected = selector.fit_transform(X_test_std, Y_test)
print(X_train_std.shape)
print(X_train_selected.shape)

(393, 8)
(393, 7)


#### 4) Comparación de modelos de clasificación mediante validación cruzada (3 puntos)
Instrucciones:

- Aplicar una validación cruzada interna de K=5 bolsas para optimizar y comparar la capacidad predictiva de los siguientes modelos: Regresión Logística y Support Vector Machine.
- La optimización de hiperparámetros debe realizarse de manera automática. (Recordad la función "GridSearchCV").
- La comparación debe realizarse únicamente en términos de exactitud proporcionando resultados de media +- desviación estándar.

In [ ]:
algoritmos = { "LOGR": LogisticRegression(random_state=42),
              "SVM": SVC(random_state=42)
         }
scores = {}
bag_size = 5
scorer = metrics.make_scorer(metrics.accuracy_score)
for name, m in algoritmos.items():
    scores[name] = cross_val_score(m, X_train_selected, Y_train, cv=KFold(n_splits=bag_size, shuffle=True, random_state=42), scoring=scorer)
    print(f"{name}: Exactitud: {scores[name].mean()} +- {scores[name].std()}")
    
    

LOGR: Exactitud: 0.6765660499837715 +- 0.05376587102746887
SVM: Exactitud: 0.6841934436871145 +- 0.04911119734717438


#### 5) Evaluación de los modelos sobre el conjunto de test (2.5 puntos)
- Entrenar los modelos anteriores utilizando todos los datos de entrenamiento.
- Evaluar su rendimiento sobre el conjunto de test mostrando una tabla de resultados tal que:
 * Las filas serán: Precisión, Sensibilidad, F-score, Exactitud y AUC
 * Las columnas serán: LOGR y SVM

In [ ]:
ypred = {}
metricas = { "PRECISION": lambda y_true, y_pred:
            metrics.precision_score(y_true, y_pred, average='micro'),
            "SENSIBILIDAD": lambda y_true, y_pred:
            metrics.recall_score(y_true, y_pred, average='micro'),
            "F1SCORE":lambda y_true, y_pred:
            metrics.f1_score(y_true, y_pred, average='micro'),
            "EXACTITUD": metrics.accuracy_score,
            "AUC": metrics.roc_auc_score}
metrics_per_alg = {}
ypred = {}

for name, alg in algoritmos.items():
    print(name)
    modelo = alg.fit(X_train_selected, Y_train)
    ypred[name] = modelo.predict(X_test_selected)
    results = {}
    for metric_name, m in metricas.items():
        results[metric_name] = m(Y_test, ypred[name])
    metrics_per_alg[name] = results


print("METRIC", "LOGR", "SVM")
for key, _ in metrics_per_alg["LOGR"].items():
    print(key, metrics_per_alg["LOGR"][key], metrics_per_alg["SVM"][key])


LOGR
SVM
METRIC LOGR SVM
PRECISION 0.7070707070707071 0.7171717171717171
SENSIBILIDAD 0.7070707070707071 0.7171717171717171
F1SCORE 0.7070707070707071 0.7171717171717171
EXACTITUD 0.7070707070707071 0.7171717171717171
AUC 0.7077205882352942 0.7138480392156862


#### 6) Visualización de resultados (1 punto)
- Mostrar la matriz de confusión
- Mostrar en una única figura la comparación de las curvas ROC obtenidas por cada modelo. (Recordad que SVM requiere un parámetro específico en su llamada para poder calcular la curva ROC). 

In [ ]:
print(metrics.confusion_matrix(Y_test, ypred["LOGR"]))
print(metrics.confusion_matrix(Y_test, ypred["SVM"]))

[[35 16]
 [13 35]]
[[42  9]
 [19 29]]


#### 7) Interpretación de resultados (0.5 puntos)
* Justifica brevemente cuál de los dos modelos utilizarías para ponerlo en producción